In [4]:
import cv2
import numpy as np
import pyaudio
import random
import pygame
import time

# Inisialisasi pygame untuk backsound
pygame.mixer.init()
background_music = pygame.mixer.Sound("background_music.wav")  # Ganti nama file jika perlu

# Setup Kamera
cap = cv2.VideoCapture(0)

# Setup PyAudio
p = pyaudio.PyAudio()
rate = 44100
chunk = 1024
stream = p.open(format=pyaudio.paInt16, channels=1, rate=rate, input=True, frames_per_buffer=chunk)

# Load karakter dan atur ukuran
character_img = cv2.imread("supermario.png", cv2.IMREAD_UNCHANGED)
character_jump_img = cv2.imread("supermario.png", cv2.IMREAD_UNCHANGED)  # Tambahan untuk animasi lompat
character_scale = 0.15
char_resized = False

# Variabel kontrol permainan
speed = 5
jumping = False
y_velocity = 0
gravity = 1
score = 0
previous_pitch = 0
jump_threshold = 1000
game_over = False
game_started = False
game_start_time = None

# Ground blocks
ground_blocks = []

# Fungsi

def get_pitch(audio_data):
    return np.mean(np.abs(audio_data))

def generate_ground_block(frame_w, frame_h):
    block_width = random.randint(100, 150)
    block_height = 20
    gap = random.randint(150, 200)
    x = frame_w + gap
    y = random.randint(int(frame_h * 0.7), int(frame_h * 0.8))
    block_type = random.choice(["platform", "gap", "rock"])
    return {"x": x, "y": y, "w": block_width, "h": block_height, "type": block_type}

def reset_game(frame_h, frame_w):
    global jumping, y_velocity, score, previous_pitch, game_over, game_started, ground_blocks, y, x, game_start_time
    jumping = False
    y_velocity = 0
    score = 0
    previous_pitch = 0
    game_over = False
    game_started = True
    ground_blocks = []
    y = int(frame_h * 0.65)

    # Tambahkan beberapa blok awal sebagai platform aman
    for i in range(4):
        block = {
            "x": i * 160,
            "y": int(frame_h * 0.75),
            "w": 140,
            "h": 20,
            "type": "platform"
        }
        ground_blocks.append(block)

    # Tambahkan blok acak setelah blok aman
    for i in range(4, 8):
        block = generate_ground_block(frame_w, frame_h)
        block["x"] = i * 160
        ground_blocks.append(block)

    # Tempatkan karakter di atas platform pertama
    first_platform = None
    for block in ground_blocks:
        if block["type"] == "platform":
            first_platform = block
            break

    x = frame_w // 3
    if first_platform:
        y = first_platform["y"] - int(frame_h * character_scale)
    else:
        y = int(frame_h * 0.65)  # fallback

    game_start_time = time.time()

# Start
ret, frame = cap.read()
frame_h, frame_w = frame.shape[:2]

# Resize karakter
if not char_resized:
    character = cv2.resize(character_img, (int(frame_w * character_scale), int(frame_w * character_scale)))
    character_jump = cv2.resize(character_jump_img, (int(frame_w * character_scale), int(frame_w * character_scale)))
    character_height, character_width = character.shape[:2]
    char_resized = True

# Posisi tetap karakter
x = frame_w // 3
y = int(frame_h * 0.65)

reset_game(frame_h, frame_w)
background_music.play(-1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Audio input
    audio_data = np.frombuffer(stream.read(chunk), dtype=np.int16)
    pitch = get_pitch(audio_data)

    # Gerak ground jika suara konstan ATAU karakter sedang melompat
    if pitch < 500 or jumping:
        for block in ground_blocks:
            block["x"] -= speed
    elif pitch - previous_pitch > jump_threshold and not jumping:
        jumping = True
        y_velocity = -15

    # Update lompatan
    if jumping:
        y += y_velocity
        y_velocity += gravity

    # Tambahkan blok baru jika perlu
    if ground_blocks and ground_blocks[-1]["x"] < frame_w:
        ground_blocks.append(generate_ground_block(frame_w, frame_h))

    # Hapus blok yang keluar
    ground_blocks = [b for b in ground_blocks if b["x"] + b["w"] > 0]

    # Cek collision
    on_ground = False
    for block in ground_blocks:
        if block["type"] == "gap":
            continue
        if block["x"] < x + character_width and x < block["x"] + block["w"]:
            if y + character_height >= block["y"] and y + character_height <= block["y"] + block["h"]:
                y = block["y"] - character_height
                jumping = False
                on_ground = True
                if block["type"] == "platform":
                    score += 1
                break

    if not on_ground and not jumping:
        if time.time() - game_start_time > 1.5:
            background_music.stop()
            game_over = True

    # Gambar ground
    for block in ground_blocks:
        color = (0, 255, 0) if block["type"] == "platform" else (128, 128, 128)
        if block["type"] != "gap":
            cv2.rectangle(frame, (block["x"], block["y"]), (block["x"] + block["w"], block["y"] + block["h"]), color, -1)

    # Gambar karakter (animasi lompat)
    current_sprite = character_jump if jumping else character
    if y + character_height <= frame_h and x + character_width <= frame_w:
        roi = frame[y:y+character_height, x:x+character_width]
        gray_char = cv2.cvtColor(current_sprite, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray_char, 1, 255, cv2.THRESH_BINARY)
        bg = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
        fg = cv2.bitwise_and(current_sprite, current_sprite, mask=mask)
        dst = cv2.add(bg, fg)
        frame[y:y+character_height, x:x+character_width] = dst

    # Tampilkan skor
    cv2.putText(frame, f"Score: {score}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Tampilkan frame
    cv2.imshow("ScreamGo Hero", frame)

    previous_pitch = pitch

    if game_over:
        cv2.putText(frame, "GAME OVER", (frame_w//2 - 100, frame_h//2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
        cv2.imshow("ScreamGo Hero", frame)
        key = cv2.waitKey(0)
        if key == ord(' '):
            reset_game(frame_h, frame_w)
        else:
            break

    if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("ScreamGo Hero", cv2.WND_PROP_VISIBLE) < 1:
        break

# Bersih-bersih
stream.stop_stream()
stream.close()
p.terminate()
cap.release()
cv2.destroyAllWindows()